# Flatten to a single list of post

In [1]:
import json
from IPython.display import display, HTML
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from pprint import pprint
import numpy as np
def flatten(z):
   return [x for y in z for x in y]

# with open('data/xaa') as f:
#     for line in f:
#         data = json.load(line)
#         print(data)
#     posts_list = map(lambda x: x['posts'], data)
#     posts_list_f = flatten(posts_list)
#     pprint(flatten(posts_list))
#     message_list = map(lambda x: x[1], flatten(posts_list))
#     train = pd.DataFrame(posts_list_f, columns=['timestamp','untokenized_post'])
#     display(message_list)

contents = open('data/xaa', "r").read() 
data = [json.loads(str(item)) for item in contents.strip().split('\n')]
user_list = map(lambda x: x[0]['posts'], data)
post_list = flatten(user_list)
posts_df = pd.DataFrame(post_list, columns=['timestamp','untokenized_post'])

display(posts_df.head(10))







,timestamp,untokenized_post
0,1376262679,"I can't fucking read the signs of the protestors, maybe it's just the res on my phone app..."
1,1376278407,No people say km for kilometres not k
2,1376374810,Yesssss KOTOR just bought both for like $2 on steam during the sale and replaying :D
3,1376378867,Oh my fuck the cats face in the corners
4,1376438053,"Do you spend much time on places like Reddit? How does it feel seeing Bryan and yourself and other Breaking Bad references come up so often, bitch?"
5,1376448214,That's the point....
6,1376728724,Good karma...hehe
7,1376729322,Just bought for $2 on steam thanks to the summer sale and am currently replaying :)
8,1376804428,5 food groups? It's fruits and vegetables together....
9,1376822284,You're what's fucking wrong with the world


# Lexicon Embeddings

In [2]:
from nltk.tokenize import TweetTokenizer, WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
import string
twitter_tknzr = TweetTokenizer()
wp_tknzr = WordPunctTokenizer()

In [3]:
# Check if b is a substring of a
def contains(a, b):
    if b in a:
        return True
    else:
        return False
# Check if elem exists in a lst
def exists(lst, elem):
    return next((1 for x in lst if contains(x,elem)), 0)


In [4]:
# _words = single words, check with exact match against a word
# _phrases = words or phrases, checked with substring against the whole sentence

# Read in NRC Lexicon
nrc_emo_words = pd.read_csv('sentiment_lexicons/en/single_word/nrc-emotion-words-v0.92.txt', sep="\t", names=["word","emotion","isEmotion"])
nrc_pos_words = nrc_emo_words[(nrc_emo_words.emotion == 'positive') & (nrc_emo_words.isEmotion == 1)]
nrc_pos_words_list = list(nrc_pos_words.word)
nrc_neg_words = nrc_emo_words[(nrc_emo_words.emotion == 'negative') & (nrc_emo_words.isEmotion == 1)]
nrc_neg_words_list = list(nrc_neg_words.word)

# Read in BingLiu Lexicon
bingliu_pos_words = pd.read_csv('sentiment_lexicons/en/single_word/bingliu-positive-words.txt', names=["word"])
bingliu_pos_words_list = list(bingliu_pos_words.word)
bingliu_neg_words = pd.read_csv('sentiment_lexicons/en/single_word/bingliu-negative-words.txt', names=["word"])
bingliu_neg_words_list = list(bingliu_neg_words.word)

# Read in HowNet Lexicon
hn_subj_phrases = pd.read_csv('sentiment_lexicons/en/phrase/hn-subjective-words-modified.txt', names=["word"], skiprows=1)
hn_subj_phrases_list = list(hn_subj_phrases.word)

hn_neg_phrases = pd.read_csv('sentiment_lexicons/en/phrase/hn-negative-words.txt', names=["word"], skiprows=1)
hn_neg_phrases_list = list(hn_neg_phrases.word)

hn_deg_adv_phrases = pd.read_csv('sentiment_lexicons/en/phrase/hn-degree-adverb-words-modified.txt', names=["word"], skiprows=1)
hn_deg_adv_phrases_list = list(hn_deg_adv_phrases.word)

# Read in MPQA Subjectivity Lexicon
fs = open('sentiment_lexicons/en/phrase/mpqa-subjectivity-lexicon.tff').read()
lines = [x for x in fs.strip().split('\n')]
def parseKeyValuePair(line):
    return dict([kv_pair.split('=') for kv_pair in line.split(' ')])
mpqa_subj_words = pd.DataFrame(list(map(parseKeyValuePair, lines)))
mpqa_weak_subj_words = mpqa_subj_words[(mpqa_subj_words.type == 'weaksubj')]
# NOTE: Take strong subjectivity words as "Subjective"
mpqa_strong_subj_words = mpqa_subj_words[(mpqa_subj_words.type == 'strongsubj')]
mpqa_strong_subj_words_list = list(mpqa_strong_subj_words.word1)

# mpqa_subj_phrases = pd.read_csv('sentiment_lexicons/en/phrase/mpqa-subjectivity-lexicon.tff')
# display(mpqa_subj_phrases)

# Compile list of words
# Positive sentiment
pos_sent_words_list = nrc_pos_words_list + bingliu_pos_words_list
# Negative sentiment
neg_sent_words_list = nrc_neg_words_list + bingliu_neg_words_list
# Subjective phrases
subj_phrases_list = mpqa_strong_subj_words_list
# Negative word
neg_phrases_list = hn_neg_phrases_list
# Degree adverb word
deg_adv_phrases_list = hn_deg_adv_phrases_list



In [11]:
from timeit import default_timer as timer
start = timer()
import concurrent.futures
def getLexiconVector(text):
    tokens = wp_tknzr.tokenize(text)
    # tag POS
    tokens_tagged = nltk.pos_tag(tokens)
    stop_words = set(stopwords.words('english'))
    # remove stopwords, filter punctuation
    words_tagged = [(word_tagged[0].lower(), word_tagged[1]) for word_tagged in tokens_tagged if (not word_tagged[0] in stop_words) and word_tagged[0].isalpha() ]
    lex_vecs = []
    for word_tagged in words_tagged:
        lex_vec = np.zeros([1,10])
        # positive
        lex_vec[0][0] = exists(pos_sent_words_list, word_tagged[0])
        # negative
        lex_vec[0][1] = exists(neg_sent_words_list, word_tagged[0])
        # neutral
        lex_vec[0][2] = not(lex_vec[0][0]) and not(lex_vec[0][1])
        # objective
        lex_vec[0][3] = not(exists(subj_phrases_list, word_tagged[0]))
        # negative
        lex_vec[0][4] = exists(neg_phrases_list, word_tagged[0])
        # degree adverb
        lex_vec[0][5] = exists(deg_adv_phrases_list, word_tagged[0])
        # noun
        lex_vec[0][6] = word_tagged[1].startswith('NN')
        # verb
        lex_vec[0][7] = word_tagged[1].startswith('VB')
        # adjective
        lex_vec[0][8] = word_tagged[1].startswith('JJ')
        # adverb
        lex_vec[0][9] = word_tagged[1].startswith('RB')
        lex_vecs.append(lex_vec)
    if len(lex_vecs) == 0:
        return []
    else:
        return np.stack(lex_vecs).reshape(len(lex_vecs), 10, 1)

# sentence = "Good karma...hehe"
# getLexiconVector(sentence)
with concurrent.futures.ProcessPoolExecutor() as executor:
    count = 0
    records = list(posts_df.untokenized_post
                  )
    feat_matrix_list = list(executor.map(getLexiconVector, records))
    print("Processed", len(feat_matrix_list), "records")
    duration = timer() - start
    print(duration)
            
# lex_vec_list = [getLexiconVector(post) for post in list(posts_df.untokenized_post.head(1000))]
# for lex_vec in lex_vec_list:
#     print(lex_vec.shape)



Processed 13048 records
423.2522923719953


In [4]:
# 1 x 10 matrix
# +ve sentiment
# -ve sentiment
# neutral
# objective
# negative
# degree adverb
# noun
# verb
# adjective
# adverb
a = np.zeros(shape=(10,1))



NameError: name 'np' is not defined